In [2]:
import duckdb
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import osmnx as ox
import sqlparse

C:\Users\dune\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
# read poi_uk.gpkg
poi_uk = gpd.read_file('data/poi_uk/poi_uk.gpkg')

In [6]:
poi_uk

,id,primary_name,main_category,alternate_category,address,locality,postcode,region,country,source,source_record_id,lat,long,LSOA21NM,LSOA21CD,NI_DZ2021CD,SG_DZ2011CD,h3_15,geometry
0,08f18729136085440362dcd6d1bc70c8,"""Crim Rocks""","""landmark_and_historical_building""",island|fountain,None,None,None,None,None,"""meta""","""132797990092265""",-6.450000,49.896700,None,None,None,None,8f7b06288550a1e,POINT (-6.45000 49.89670)
1,08f187291377252c03ada238ea2e2308,"""Zantman's Rock""","""landmark_and_historical_building""",island|fountain,None,None,None,None,None,"""meta""","""129879807053615""",-6.447500,49.899400,None,None,None,None,8f7b0628858855b,POINT (-6.44750 49.89940)
2,08f187290305a2db036e8c44b2ca7b73,"""Troytown Farm & Campsite""","""campground""",farm|dairy_farm,"""Troytown Farm, St Agnes""",None,"""TR22 0PL""",None,"""GB""","""meta""","""291582648427""",-6.353102,49.890598,None,None,None,None,8f7b02931219440,POINT (-6.35310 49.89060)
3,08f18729030624ee032a3a8a746fde6a,"""St Agnes Watersports""",None,None,"""Hellweathers""",None,"""TR22 0PL""",None,"""GB""","""meta""","""199148563923007""",-6.351283,49.892632,None,None,None,None,8f7b02931283cb6,POINT (-6.35128 49.89263)
4,08f187290edb320c03723fb73fdb4ced,"""Wingletang Down""","""landmark_and_historical_building""",national_park|monument,None,"""Hugh Town""",None,None,"""GB""","""meta""","""196677457129876""",-6.341000,49.888000,None,None,None,None,8f7b0290612b341,POINT (-6.34100 49.88800)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426297,08f09a241e8cccb6034c398be46ed9f6,"""Harding oilfield""","""landmark_and_historical_building""",structure_and_geography,None,None,None,None,None,"""meta""","""119016228188419""",1.514412,59.278991,None,None,None,None,8f8523614801644,POINT (1.51441 59.27899)
2426298,08f09a239ad2589d0305891298fa53f6,"""Bruce Gas Field""","""landmark_and_historical_building""",attractions_and_activities|fountain,None,None,None,None,None,"""meta""","""597185257115445""",1.675000,59.741670,None,None,None,None,8f8505964216948,POINT (1.67500 59.74167)
2426299,08f09a260a26b8f1030812ebae61d459,"""Alvheim FPSO""","""business_manufacturing_and_supply""",None,None,"""Kristiansund""",None,None,"""NO""","""meta""","""416120492449823""",1.996722,59.567316,None,None,None,None,8f8523adc6b3504,POINT (1.99672 59.56732)
2426300,08f09a3b11a6e084038b4d9e2986ef31,"""Ninian South Platform""","""health_and_medical""",jail_and_prison|hotel,None,"""Glendale""",None,"""CA""","""US""","""meta""","""342762022512541""",1.449388,60.804876,None,None,None,None,8f852a10811bcde,POINT (1.44939 60.80488)


In [3]:
#Get London Bounding Box

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi")

# Get location data for London
location = geolocator.geocode("London")

# Get the bounding box
bounding_box_london = location.raw['boundingbox']

min_lat_london, max_lat_london = float(bounding_box_london[0]), float(bounding_box_london[1])
min_lon_london, max_lon_london = float(bounding_box_london[2]), float(bounding_box_london[3])

print("London Bounding Box Coordinates:")
print(f"Min Longitude: {min_lon_london}")
print(f"Min Latitude: {min_lat_london}")
print(f"Max Longitude: {max_lon_london}")
print(f"Max Latitude: {max_lat_london}")
print()

London Bounding Box Coordinates:
Min Longitude: -0.5103751
Min Latitude: 51.2867601
Max Longitude: 0.3340155
Max Latitude: 51.6918741



In [7]:
#Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

#Test new config
ox.config(use_cache=True, log_console=True)

buildings = ox.features_from_place(place_name, tags={'building': True})
buildings = buildings[buildings.geometry.notnull()]
building_footprints = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in building_footprints.columns:
    if building_footprints.apply(lambda x: isinstance(x, list)).any():
        building_footprints = building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

building_footprints = building_footprints[['name', 'geometry']].reset_index()
print(building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
building_footprints.to_file("data/osm_data/boroughs_buildings.geojson", driver="GeoJSON")

C:\Users\dune\AppData\Local\Temp\ipykernel_9948\2344882920.py:5: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(use_cache=True, log_console=True)


MemoryError: Unable to allocate 13.9 GiB for an array with shape (1780, 1049612) and data type object

In [ ]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

buildings_city = ox.features_from_place(place_name, tags={'building': True})
buildings_city = buildings_city[buildings_city.geometry.notnull()]
building_footprints_city = buildings_city[buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in building_footprints_city.columns:
    if building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        building_footprints_city = building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

building_footprints_city = building_footprints_city[['name', 'geometry']].reset_index()

building_footprints_combined = pd.concat([building_footprints, building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
building_footprints_combined.to_file("data/osm_data/all_london_buildings.geojson", driver="GeoJSON")

In [ ]:
#Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

commercial_buildings = ox.features_from_place(place_name, tags={'building': ['commercial']})

commercial_buildings = commercial_buildings[commercial_buildings.geometry.notnull()]
commercial_building_footprints = commercial_buildings[commercial_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in commercial_building_footprints.columns:
    if commercial_building_footprints.apply(lambda x: isinstance(x, list)).any():
        commercial_building_footprints = commercial_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

commercial_building_footprints = commercial_building_footprints[['name', 'geometry']].reset_index()
print(commercial_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
commercial_building_footprints.to_file("data/osm_data/boroughs_commercial_buildings.geojson", driver="GeoJSON")

     element_type     osmid                      name  \
0             way   4076571  Bromley Road Retail Park   
1             way   4076587   Catford Delivery Office   
2             way   5790535              Refuge House   
3             way   5986754         One Canada Square   
4             way   5987126                       NaN   
...           ...       ...                       ...   
5294     relation  13472609                Hyde House   
5295     relation  13535999           Old Guy's House   
5296     relation  15685271                       NaN   
5297     relation  15761586               China Works   
5298     relation  16030676           Camelford House   

                                               geometry  
0     POLYGON ((-0.02063 51.43669, -0.02047 51.43631...  
1     POLYGON ((-0.02001 51.43859, -0.02004 51.43865...  
2     POLYGON ((-0.07800 51.65301, -0.07777 51.65303...  
3     POLYGON ((-0.02002 51.50477, -0.01996 51.50500...  
4     POLYGON ((-0.02588 

In [ ]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

commercial_buildings_city = ox.features_from_place(place_name, tags={'building': ['commercial']})
commercial_buildings_city = commercial_buildings_city[commercial_buildings_city.geometry.notnull()]
commercial_building_footprints_city = commercial_buildings_city[commercial_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in commercial_building_footprints_city.columns:
    if commercial_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        commercial_building_footprints_city = commercial_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

commercial_building_footprints_city = commercial_building_footprints_city[['name', 'geometry']].reset_index()

commercial_building_footprints_combined = pd.concat([commercial_building_footprints, commercial_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
commercial_building_footprints_combined.to_file("data/osm_data/all_london_commercial_buildings.geojson", driver="GeoJSON")

### Extract Building Type Info from OSM - Office

In [ ]:
# Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

ox.config(use_cache=True, log_console=True)

office_buildings = ox.features_from_place(place_name, tags={'building': ['office']})

office_buildings = office_buildings[office_buildings.geometry.notnull()]
office_building_footprints = office_buildings[office_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in office_building_footprints.columns:
    if office_building_footprints.apply(lambda x: isinstance(x, list)).any():
        office_building_footprints[col] = office_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

office_building_footprints = office_building_footprints[['name', 'geometry']].reset_index()
print(office_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
office_building_footprints.to_file("data/osm_data/boroughs_office_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/4117539463.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


     element_type     osmid                       name  \
0             way   4237383                        ITN   
1             way   5251705                        NaN   
2             way   5986805                    HSBC UK   
3             way   5986920                       KPMG   
4             way   5986929                   Barclays   
...           ...       ...                        ...   
1724     relation  12566775                        NaN   
1725     relation  12581563                BBC Studios   
1726     relation  14168517  The Featherstone Building   
1727     relation  15690226                 The Hudson   
1728     relation  16030675             Tintagel House   

                                               geometry  
0     POLYGON ((-0.11445 51.52304, -0.11478 51.52344...  
1     POLYGON ((-0.23366 51.40761, -0.23281 51.40822...  
2     POLYGON ((-0.01792 51.50522, -0.01794 51.50525...  
3     POLYGON ((-0.01680 51.50508, -0.01663 51.50569...  
4     POLYGON

In [ ]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

office_buildings_city = ox.features_from_place(place_name, tags={'building': ['office']})
office_buildings_city = office_buildings_city[office_buildings_city.geometry.notnull()]
office_building_footprints_city = office_buildings_city[office_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in office_building_footprints_city.columns:
    if office_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        office_building_footprints_city = office_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

office_building_footprints_city = office_building_footprints_city[['name', 'geometry']].reset_index()

office_building_footprints_combined = pd.concat([office_building_footprints, office_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
office_building_footprints_combined.to_file("data/osm_data/all_london_office_buildings.geojson", driver="GeoJSON")

### Extract Building Type Info from OSM - Residential

In [ ]:
# Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

ox.config(use_cache=True, log_console=True)

residential_buildings = ox.features_from_place(place_name, tags={'building': ['residential']})

residential_buildings = residential_buildings[residential_buildings.geometry.notnull()]
residential_building_footprints = residential_buildings[residential_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in residential_building_footprints.columns:
    if residential_building_footprints.apply(lambda x: isinstance(x, list)).any():
        residential_building_footprints[col] = residential_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

residential_building_footprints = residential_building_footprints[['name', 'geometry']].reset_index()
print(residential_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
residential_building_footprints.to_file("data/osm_data/boroughs_residential_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/160667686.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


      element_type     osmid            name  \
0              way   4948114    Kemble House   
1              way   4948116  Kettleby House   
2              way   4948118   Woolley House   
3              way   5131193             NaN   
4              way   5131194             NaN   
...            ...       ...             ...   
83799     relation  16389021     Birch House   
83800     relation  16389022  Lansdell House   
83801     relation  16847821             NaN   
83802     relation  17193995             NaN   
83803     relation  17729875             NaN   

                                                geometry  
0      POLYGON ((-0.10656 51.46576, -0.10645 51.46582...  
1      POLYGON ((-0.10571 51.46618, -0.10559 51.46624...  
2      POLYGON ((-0.10486 51.46659, -0.10473 51.46665...  
3      POLYGON ((-0.19992 51.46981, -0.19992 51.46987...  
4      POLYGON ((-0.19955 51.46940, -0.19955 51.46980...  
...                                                  ...  
83799  POL

In [ ]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

residential_buildings_city = ox.features_from_place(place_name, tags={'building': ['residential']})
residential_buildings_city = residential_buildings_city[residential_buildings_city.geometry.notnull()]
residential_building_footprints_city = residential_buildings_city[residential_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in residential_building_footprints_city.columns:
    if residential_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        residential_building_footprints_city = residential_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

residential_building_footprints_city = residential_building_footprints_city[['name', 'geometry']].reset_index()

residential_building_footprints_combined = pd.concat([residential_building_footprints, residential_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
residential_building_footprints_combined.to_file("data/osm_data/all_london_residential_buildings.geojson", driver="GeoJSON")

### Extract Building Type Info from OSM - Retail

In [ ]:
# Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

ox.config(use_cache=True, log_console=True)

retail_buildings = ox.features_from_place(place_name, tags={'building': ['retail']})

retail_buildings = retail_buildings[retail_buildings.geometry.notnull()]
retail_building_footprints = retail_buildings[retail_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in retail_building_footprints.columns:
    if retail_building_footprints.apply(lambda x: isinstance(x, list)).any():
        retail_building_footprints[col] = retail_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

retail_building_footprints = retail_building_footprints[['name', 'geometry']].reset_index()
print(retail_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
retail_building_footprints.to_file("data/osm_data/boroughs_retail_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/110674602.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


      element_type     osmid                                            name  \
0              way   4270351                                             NaN   
1              way   4425414                                             NaN   
2              way   4425469                                             NaN   
3              way   4770296  Kingston Rowing Club / Tiffin School Boat Club   
4              way   4775619                              St Richard's Court   
...            ...       ...                                             ...   
11690     relation  14208162                   Gulf Ickenham Service Station   
11691     relation  15536955                                             NaN   
11692     relation  16095623                                         Crispin   
11693     relation  17367661                                             NaN   
11694     relation  17745415             Ilford Tesco Superstore Barkingside   

                                       

In [ ]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

retail_buildings_city = ox.features_from_place(place_name, tags={'building': ['retail']})
retail_buildings_city = retail_buildings_city[retail_buildings_city.geometry.notnull()]
retail_building_footprints_city = retail_buildings_city[retail_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in retail_building_footprints_city.columns:
    if retail_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        retail_building_footprints_city = retail_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

retail_building_footprints_city = retail_building_footprints_city[['name', 'geometry']].reset_index()

retail_building_footprints_combined = pd.concat([retail_building_footprints, retail_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
retail_building_footprints_combined.to_file("data/osm_data/all_london_retail_buildings.geojson", driver="GeoJSON")